# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
N. Wang  ->  N. Wang  |  ['N. Wang']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Arxiv has 62 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.08903


extracting tarball to tmp_2601.08903... done.


S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Issues with the citations
plugin pybtex.database.input.suffixes for suffix .tex not found
Retrieving document from  https://arxiv.org/e-print/2601.09054


extracting tarball to tmp_2601.09054...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.09521
extracting tarball to tmp_2601.09521...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.09587


extracting tarball to tmp_2601.09587... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.08903-b31b1b.svg)](https://arxiv.org/abs/2601.08903) | **IK Pegasi and the Double Merger Path to Type Ia Supernovae**  |
|| N. Hallakoun, et al. -- incl., <mark>S. Shahaf</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2026-01-15*|
|*Comments*| *ApJL, in press*|
|**Abstract**|            Recent Gaia astrometry has revealed thousands of main-sequence + white-dwarf binaries (MS+WD) at separations of ~0.1-10 au, including a subset hosting unusually massive (>~0.8 Msun) WDs. We argue that s-process enrichment in the non-degenerate companion provides a powerful diagnostic for identifying WDs that formed via mergers in hierarchical triple systems. For a massive WD, standard single-star evolution requires a massive (>~4 Msun) progenitor, yet such progenitors produce negligible s-process yields. We define IK Peg-type systems as those exhibiting this mass-yield tension: barium-enhanced companions orbiting WDs too massive to have descended from efficient s-process producers. The well-known system IK Peg exemplifies this class. Applying this framework to published spectroscopic data reveals several additional candidates, and we estimate that a few dozen such systems should exist in the current Gaia sample. If these systems trace inner-binary mergers in primordial triples, they represent observable intermediate stages towards eventual Type Ia supernovae via the double-merger pathway, as predicted by recent population-synthesis models.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.09587-b31b1b.svg)](https://arxiv.org/abs/2601.09587) | **The Structure and Kinematics of Three Class 0 Protostellar Jets from JWST**  |
|| S. A. Federman, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2026-01-15*|
|*Comments*| *25 pages, 15 figures, 7 tables*|
|**Abstract**|            We present observations of jets within 2000 au of three deeply embedded protostars using 2.9-27 micron observations with JWST. These observations show the morphologies and kinematics of the collimated jets from three protostars, the low-mass Class 0 protostars B335 and HOPS 153, and the intermediate-mass protostar HOPS 370. These jets are traced by shock-ionized fine-structure line emission observed with the JWST NIRSpec and MIRI IFUs. We find that [Fe II] emission traces the full extent of the inner 1000 to 2000 au of the jets, depending on distance to the protostar, while other ions mostly trace isolated shocked knots. The jets show evidence of wiggling motion in the plane of the sky as well as asymmetries between blue and red-shifted lobes. The widths of the jets increase non-monotonically with distance from the central protostar, with opening angles ranging from 2.1 degrees to < 10.1 degrees for the three protostars in the sample. The jets have total velocities ranging from 147 to 184 km/s after correcting for disk inclination. For B335, an 8-month gap between NIRSpec and MIRI MRS observations enabled measurement of the tangential velocity of a shocked knot; in combination with the radial velocity, this shows that the jet has a different inclination than the outflow cavity. We find multiple knots before and during a recent outburst in B335, although the knots were more frequent during the burst. The asymmetries between blue- and red-shifted lobes strongly suggest complex interactions between the circumstellar disks and magnetic fields.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.09054-b31b1b.svg)](https://arxiv.org/abs/2601.09054) | **Upper limits on microhertz gravitational waves from supermassive black-hole binaries using PSR J1909-3744 data from the second IPTA data release**  |
|| J. Zou, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2026-01-15*|
|*Comments*| *17 pages, 5 figures*|
|**Abstract**|            We present the results of a search for gravitational waves (GWs) from individual sources using high-cadence observations of PSR J1909\(-\)3744 obtained during an intensive observing campaign with the International Pulsar Timing Array second data release (IPTA-DR2) between July 2010 and November 2012. The observations, conducted at three different radio frequencies with the Nançay Radio Telescope (NRT) and Parkes Telescope (PKS) and five frequencies with the Green Bank Telescope (GBT), enabled precise corrections for dispersion measure effects and scattering variations. After these corrections, the timing residuals showed an unmodeled periodic noise component with an amplitude of 340 ns. Our analysis yields upper limits on the GW strain from individual sources, constraining it to be below \(1.9 \times 10^{-14}\) at 71 nHz and \(2.3 \times 10^{-13}\) at 1 \textmu Hz for average sky locations, while for optimal source locations the limits improve to \(6.2 \times 10^{-15}\) and \(8.9 \times 10^{-14}\) at the same frequencies, respectively. Our new limits are about a factor of 1.52 more stringent than those of Perera et al. based on an earlier EPTA data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.09521-b31b1b.svg)](https://arxiv.org/abs/2601.09521) | **Resolving Hubble tension and locating missing baryons: Synergies between fast radio bursts and emerging cosmological probes**  |
|| P.-J. Wu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-01-15*|
|*Comments*| *11 pages, 3 figures*|
|**Abstract**|            Two of the most pressing challenges in cosmology are the persistent discrepancy in measurements of the Hubble constant, referred to as the Hubble tension, and the deficit of baryons in the local Universe, known as the missing baryon problem. Fast radio bursts (FRBs) encode the integrated electron column density along their lines of sight, offering a unique probe of both the cosmic expansion rate ($H_0$) and the baryon density ($\Omega_{\rm b}$). However, constraints from FRBs alone suffer from a severe $H_0$-$\Omega_{\rm b}$ degeneracy that prevents them from resolving either problem independently. We show that this degeneracy can be broken by combining FRBs with other emerging probes whose degeneracy directions differ in the $H_0$-$\Omega_{\rm b}$ plane. Specifically, we quantify three multi-messenger approaches: FRBs paired with gravitational wave (GW) standard sirens, strong gravitational lensing (SGL) time delays, and 21 cm intensity mapping (IM). The combinations FRB+GW, FRB+SGL, and FRB+21 cm IM each deliver simultaneous constraints on $H_0$ and $\Omega_{\rm b}$ better than ($1\%$, $1\%$) in the $\Lambda$CDM model, ($1.5\%$, $2\%$) in the $w$CDM model, and ($2\%$, $3.5\%$) in the CPL model. Moreover, in a model-independent framework, both FRB+GW and FRB+SGL constrain $H_0$ and $\Omega_{\rm b}$ to better than ($1\%$, $2\%$) precision. These results demonstrate that the synergy between FRBs and other emerging probes holds great promise for resolving the Hubble tension and locating the missing baryons.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.08903/figures/Rekhi+2025_Escorza+2023_BaFe_vs_MWD_FeH.png', 'tmp_2601.08903/figures/BaFeVsMwd.png', 'tmp_2601.08903/figures/IKPegAbundances.png']
copying  tmp_2601.08903/figures/Rekhi+2025_Escorza+2023_BaFe_vs_MWD_FeH.png to _build/html/
copying  tmp_2601.08903/figures/BaFeVsMwd.png to _build/html/
copying  tmp_2601.08903/figures/IKPegAbundances.png to _build/html/
exported in  _build/html/2601.08903.md
    + _build/html/tmp_2601.08903/figures/Rekhi+2025_Escorza+2023_BaFe_vs_MWD_FeH.png
    + _build/html/tmp_2601.08903/figures/BaFeVsMwd.png
    + _build/html/tmp_2601.08903/figures/IKPegAbundances.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\rsun}{R_\odot}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# IK Pegasi and the Double Merger Path to Type Ia Supernovae

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.08903-b31b1b.svg)](https://arxiv.org/abs/2601.08903)<mark>Appeared on: 2026-01-15</mark> -  _ApJL, in press_

</div>
<div id="authors">

N. Hallakoun, et al. -- incl., <mark>S. Shahaf</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** Recent Gaia astrometry has revealed thousands of main-sequence $+$ white-dwarf binaries (MS $+$ WD) at separations of ${\sim} 0.1$ -- $10$ au, including a subset hosting unusually massive ( $\gtrsim 0.8$ $\msun$ ) WDs. We argue that _s_ -process enrichment in the non-degenerate companion provides a powerful diagnostic for identifying WDs that formed via mergers in hierarchical triple systems. For a massive WD, standard single-star evolution requires a massive ( $\gtrsim 4$ $\msun$ ) progenitor, yet such progenitors produce negligible _s_ -process yields. We define $*IK Peg--type*$ systems as those exhibiting this mass--yield tension: barium-enhanced companions orbiting WDs too massive to have descended from efficient _s_ -process producers. The well-known system IK Peg exemplifies this class. Applying this framework to published spectroscopic data reveals several additional candidates, and we estimate that a few dozen such systems should exist in the current Gaia sample. If these systems trace inner-binary mergers in primordial triples, they represent observable intermediate stages towards eventual Type Ia supernovae via the double-merger pathway, as predicted by recent population-synthesis models.

</div>

<div id="div_fig1">

<img src="tmp_2601.08903/figures/Rekhi+2025_Escorza+2023_BaFe_vs_MWD_FeH.png" alt="Fig1" width="100%"/>

**Figure 1. -** The relationship between WD mass, companion metallicity, and Ba abundance. Gaia MS+WD systems with measured Ba abundances are plotted as diamonds \citep{Rekhi_2024} and circles \citep{Rekhi_2026}. Known Ba dwarfs (pre-red giant branch; pre-RGB) hosting WD companions from \citet{Escorza_2023} are shown as triangles, while star symbols denote Ba giants. Filled symbols indicate Ba-enriched systems, defined as $\text{[Ba/Fe]} > +0.25$ for the \citet{Rekhi_2024, Rekhi_2026} samples, or classified as Ba dwarfs/strong Ba giants in the \citet{Escorza_2023} sample. The red dashed line delineates the tentative empirical limit beyond which barium enrichment is inconsistent with a single-progenitor scenario. The green dotted line marks the most conservative limit, based on single-progenitor evolution models (see Figure \ref{fig:BaFeVsMwd}). (*fig:Mwd_FeH_BaFe*)

</div>
<div id="div_fig2">

<img src="tmp_2601.08903/figures/BaFeVsMwd.png" alt="Fig2" width="100%"/>

**Figure 2. -** Maximal [Ba/Fe] abundance attained during the AGB phase for metallicities $\text{[Fe/H]} \approx -0.7$\citep{Karakas_2018}, $-0.3$, $0$, and $+0.3$\citep{Karakas_2016}, plotted against the final WD mass of a single progenitor using the \citet{Cunningham_2024} IFMR. For comparison, the positions of IK Peg (star) and the IK Peg--type candidate Gaia DR3 5550946678313327744 (diamond; \citealt{Rekhi_2026}) are indicated. Both systems have near-Solar metallicity. (*fig:BaFeVsMwd*)

</div>
<div id="div_fig3">

<img src="tmp_2601.08903/figures/IKPegAbundances.png" alt="Fig8" width="100%"/>

**Figure 8. -** IK Peg A element abundance relative to iron, from \citet[][black circles; $\text{[Fe/H]}=+0.17\pm0.17$]{Smalley_1996}, and from \citet[][red squares; $\text{[Fe/H]}=+0.07$]{Stateva_2012}. Open triangles mark upper limits. (*fig:IKPegAbundances*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.08903"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

113  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
